<a href="https://colab.research.google.com/github/Gvinay90/TensorFlow-2.0/blob/master/Transfer_Learning_and_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Where and when Transfer Learning and Fine Tunning is used?
* Large and  different dataset -> Train hole model
* Large and similar dataset -> Fine Tunning
* Small and diffrent dataset ->Fine Tunning
* Small and similar dataset -> Transfer Learning 


In [0]:
!pip install tensorflow

In [2]:
!pip install tqdm

##Dogs and Cat Dataset

### Download the dataset


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-05-25 10:25:45--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 2607:f8b0:4001:c1f::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   240MB/s    in 0.3s    

2020-05-25 10:25:45 (240 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



##Data Processing

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

## Unziping the Dogs and Cats Dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object=zipfile.ZipFile(file=dataset_path,mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

## Setting up dataset paths

In [0]:
dataset_path_new="./cats_and_dogs_filtered/"

In [0]:
train_dir=os.path.join(dataset_path_new,"train")
validation_dir=os.path.join(dataset_path_new,"validation")

## Building The Model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE=(128,128,3)

In [0]:
base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights="imagenet")

In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [0]:
base_model.trainable=False

### Defining the custom head for our network

In [15]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer= tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

### Define the model

Here we are combining two models

In [0]:
model=tf.keras.models.Model(inputs=base_model.input,outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compiling the model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=["accuracy"])

### Creating data generator

Resizing images
Big pre=trained architecture supports only certain input sizes.
for Example: MobileNet supports(96,96),(128,128),(160,160),(192,192),(224,224)

In [0]:
data_gen_train=ImageDataGenerator(rescale=1/255.)
data_gen_valid=ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator=data_gen_train.flow_from_directory(train_dir,target_size=(128,128),batch_size=128,class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator=data_gen_valid.flow_from_directory(validation_dir,target_size=(128,128),batch_size=128,class_mode="binary")

Found 1000 images belonging to 2 classes.


## Training the model

In [26]:
model.fit(train_generator,epochs=5,validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 10s 611ms/step - loss: 0.3220 - accuracy: 0.8820 - val_loss: 0.3131 - val_accuracy: 0.9000
Epoch 2/5
16/16 [==============================] - 9s 591ms/step - loss: 0.2894 - accuracy: 0.9000 - val_loss: 0.2829 - val_accuracy: 0.9100
Epoch 3/5
16/16 [==============================] - 10s 597ms/step - loss: 0.2619 - accuracy: 0.9165 - val_loss: 0.2575 - val_accuracy: 0.9160
Epoch 4/5
16/16 [==============================] - 9s 592ms/step - loss: 0.2402 - accuracy: 0.9275 - val_loss: 0.2372 - val_accuracy: 0.9240
Epoch 5/5
16/16 [==============================] - 9s 571ms/step - loss: 0.2216 - accuracy: 0.9335 - val_loss: 0.2198 - val_accuracy: 0.9290


### Transfer learning model evaluation

In [29]:
valid_loss,valid_accuracy=model.evaluate(valid_generator)

8/8 [==============================] - 3s 342ms/step - loss: 0.2198 - accuracy: 0.9290


## Fine Tunning
* No need to use fine tunning on the whole network,only few top layers are enough.The goal of the fining tunning is to adopt that specific part of the network for our custom datasets

### Unfreze a few top layers from the model

In [0]:
base_model.trainable= True

In [31]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [0]:
fine_tune_at=100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Compiling the model for fine-tunning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=['accuracy'])

### Trainning the fine tune model

In [35]:
model.fit(train_generator,epochs=5,validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 10s 633ms/step - loss: 0.1794 - accuracy: 0.9290 - val_loss: 0.0698 - val_accuracy: 0.9740
Epoch 2/5
16/16 [==============================] - 9s 579ms/step - loss: 0.0302 - accuracy: 0.9950 - val_loss: 0.0819 - val_accuracy: 0.9720
Epoch 3/5
16/16 [==============================] - 9s 585ms/step - loss: 0.0117 - accuracy: 0.9995 - val_loss: 0.0913 - val_accuracy: 0.9730
Epoch 4/5
16/16 [==============================] - 9s 588ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0946 - val_accuracy: 0.9690
Epoch 5/5
16/16 [==============================] - 9s 589ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1141 - val_accuracy: 0.9670


### Evaluating the fine tune model

In [36]:
valid_loss,valid_accuracy=model.evaluate(valid_generator)

8/8 [==============================] - 3s 337ms/step - loss: 0.1141 - accuracy: 0.9670
